In [11]:
import pandas as pd
import numpy as np
from cointegracao import Cointegration
from itertools import combinations, permutations
import time
from backtest import Executer

In [12]:
df_prices = pd.read_excel('database.xlsx', index_col=0)
portfolio = pd.read_excel('PORTFOLIO.xlsx') # Stocks in IBX for each period

In [13]:
portfolio.replace('VIVT4', 'VIVT3',inplace=True)
# Filter only portfolio stocks that are in prices database
portfolio = portfolio[portfolio.ticker.isin(df_prices.columns)]

In [14]:
df_prices.head()

,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2017-07-31,16.89,11.33,12.82,NaN,27.29,5.66,22.51,14.27,14.33,18.54,...,9.31,9.55,31.94,4.42,23.43,NaN,5.83,26.03,6.81,18.07
2017-08-01,16.97,11.41,12.88,NaN,27.19,5.78,22.98,14.34,14.51,18.58,...,9.37,9.53,32.29,4.52,23.25,NaN,5.67,25.52,6.88,17.97
2017-08-02,17.11,11.56,13.20,NaN,27.41,5.83,23.91,14.49,14.71,18.79,...,9.63,9.60,30.71,4.70,23.29,NaN,5.58,25.83,7.01,17.27
2017-08-03,16.99,11.11,13.10,NaN,28.03,5.84,24.06,14.23,14.70,18.75,...,9.57,9.99,31.09,4.72,22.97,NaN,5.67,25.86,6.89,16.34
2017-08-04,16.99,11.23,13.10,NaN,27.79,5.82,24.21,14.33,14.72,18.31,...,9.55,10.34,30.82,4.71,23.38,NaN,5.67,26.03,6.88,16.43


In [15]:
portfolio.head()

,data_ini,data_fin,ticker,setor
0,2019-07-01,2019-12-31,AZUL4,Aéreas
1,2019-07-01,2019-12-31,GOLL4,Aéreas
2,2019-07-01,2019-12-31,ABEV3,Alimentos e Bebidas
3,2019-07-01,2019-12-31,BRFS3,Alimentos e Bebidas
4,2019-07-01,2019-12-31,JBSS3,Alimentos e Bebidas


In [16]:
def create_request(df):
    
    """
    Create a dictionary with intial date, final date and 
    combination or permutation in pairs of stocks
    """
    
    # Initiates a dictionary with key as initial date 
    permut = {data_ini:[data_fin, []] for data_ini, data_fin in zip(df.data_ini.unique(), df.data_fin.unique())}

    # Loop over initial dates
    for data in df.data_ini.unique():   
        
        # Select stocks with same initial date
        df_half = df.loc[df.data_ini==data]
        
        # Loops over sectors to create a permutation
        for setor in df_half.setor.unique():
            
            # Select stocks of same sector and appends its permutations on dictionary
            tickers = df_half.loc[df_half.setor == setor, 'ticker'].to_list()
            permut[data][-1].extend(list(permutations(tickers, 2)))
    
    return permut

In [17]:
# Combinations or permutations of pairs for each halfyear
requests = create_request(portfolio)

In [18]:
coint = Cointegration(z_score_out=0.5, z_score_stop=2)
train_size = 252

In [19]:
# Loops over dict with initial, final dates and permutations
# And filters the dataframe with prices with 252 days (one year)
# of formation data and 6 months of trading data

dic_list = []

for key, value in requests.items():
    
    data_ini = key
    data_fin = value[0]

    index_ini = df_prices.index.get_indexer(target=[key], method='ffill').item()
    index_pre = df_prices.index.get_indexer(target=[data_ini], method='ffill').item() - train_size
    index_fin = df_prices.index.get_indexer(target=[value[0]], method='ffill').item()

    perm = value[1]
    
    df_trading = df_prices.iloc[index_pre: index_fin]
    
    # train_size = df_trading[data_pre:data_ini].shape[0] + 1 

    # Multithreding executer
    e = Executer(df_trading, perm, coint, train_size=train_size, twoway=False, moving_limits=False)

    if __name__ == '__main__':
        dic_list.extend(e.executer())


  0%|          | 0/286 [00:00<?, ?it/s]

  0%|          | 0/386 [00:00<?, ?it/s]

  0%|          | 0/450 [00:00<?, ?it/s]

  0%|          | 0/466 [00:00<?, ?it/s]

  0%|          | 0/484 [00:00<?, ?it/s]

  0%|          | 0/656 [00:00<?, ?it/s]

In [20]:
df_list = [pd.DataFrame(d) for d in dic_list]
df = pd.concat(df_list)

In [22]:
# df.to_csv('result_std1.csv', index=False)